<a href="https://colab.research.google.com/github/Turbantech/Object-Detection-Model/blob/main/Object_Detection_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount the Google Drive

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/gdrive/MyDrive/Detection_Dataset.rar" /content/
!apt install -y unrar
!unrar x /content/Detection_Dataset.rar /content/custom_data/

Streaming output truncated to the last 5000 lines.
Extracting  /content/custom_data/Detection_Dataset/images/f6fb20ab54b35634c87c9cfbb1d9c037_jpg.rf.89bd3d426d6b38fb86fe8b39a4948ead.jpg      58%  OK 
Extracting  /content/custom_data/Detection_Dataset/images/f7453501-myself_94.jpg      58%  OK 
Extracting  /content/custom_data/Detection_Dataset/images/f7a92965-bb8c-5ba7-b660-91a7729a9cac_jpeg.rf.f3e11bd743ade1cf63c0b1c508012625.jpg      58%  OK 
Extracting  /content/custom_data/Detection_Dataset/images/f7c1ed95-cdae-59fa-9481-3287d1aa47c7_jpeg.rf.ee1b890012cfab49ff133d7d560f6562.jpg      58%  OK 
Extracting  /content/custom_data/Detection_Dataset/images/f7c6d465-8401-573b-9fed-f47ebf93fe06_jpeg.rf.8863948f444a2a563e1571c84fe52c33.jpg      58%  OK 
Extracting  /content/custom_data/Detection_Dataset/images/f806dc7736d71a985f6fd7e0d42e438b_png.rf.bac5cf6b47f2db2d83674f790b98e5db.jpg      58%  OK 
Extracting  /content/custom_data/Detecti

Split the Dataset

In [ ]:
import os
import random
import shutil
from pathlib import Path

# ✅ Define source paths
images_dir = Path("/content/custom_data/Detection_Dataset/images")
labels_dir = Path("/content/custom_data/Detection_Dataset/labels")
output_dir = Path("/content/yolo_dataset")
train_split = 0.9

# ✅ Collect all images that have matching labels
image_exts = [".jpg", ".jpeg", ".png"]
all_images = [f for f in images_dir.iterdir() if f.suffix.lower() in image_exts]

valid_images = []
for img in all_images:
    label_file = labels_dir / (img.stem + ".txt")
    if label_file.exists():
        valid_images.append((img, label_file))

# ✅ Split
random.shuffle(valid_images)
split_index = int(len(valid_images) * train_split)
train_set = valid_images[:split_index]
val_set = valid_images[split_index:]

# ✅ Copy to YOLO format folders
def copy_files(pairs, subset):
    img_out = output_dir / subset / "images"
    lbl_out = output_dir / subset / "labels"
    img_out.mkdir(parents=True, exist_ok=True)
    lbl_out.mkdir(parents=True, exist_ok=True)

    for img_path, lbl_path in pairs:
        shutil.copy(img_path, img_out / img_path.name)
        shutil.copy(lbl_path, lbl_out / lbl_path.name)

copy_files(train_set, "train")
copy_files(val_set, "validation")

# ✅ Summary
print(f"✅ Done! Total matched pairs: {len(valid_images)}")
print(f"🟢 Train: {len(train_set)}")
print(f"🔵 Validation: {len(val_set)}")
print(f"📂 Ready at: {output_dir}")


✅ Done! Total matched pairs: 3315
🟢 Train: 2983
🔵 Validation: 332
📂 Ready at: /content/yolo_dataset


In [ ]:
data_yaml = """
path: /content/yolo_dataset
train: train/images
val: validation/images
nc: 5
names: [Laptop, Bottle, Mouse, Watch, Ishpreet]
"""

with open("/content/data.yaml", "w") as f:
    f.write(data_yaml.strip())

print("✅ data.yaml created at /content/data.yaml")


✅ data.yaml created at /content/data.yaml


In [ ]:
!yolo detect train data=/content/data.yaml model=yolov8n.pt epochs=25 imgsz=640

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
100% 6.25M/6.25M [00:00<00:00, 131MB/s]
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, i

In [ ]:
from ultralytics import YOLO

# Load the trained model
model = YOLO("runs/detect/train/weights/best.pt")  # or your path

# Run validation on validation set
metrics = model.val(data="data.yaml")

# Get key metrics from metrics.box
mean_precision = metrics.box.p.mean()
mean_recall = metrics.box.r.mean()
map_50 = metrics.box.map50
map_50_95 = metrics.box.map

# Print them nicely
print("\n📊 Evaluation Metrics:")
print(f"Mean Precision   : {mean_precision:.3f}")
print(f"Mean Recall      : {mean_recall:.3f}")
print(f"mAP@0.5          : {map_50:.3f}")
print(f"mAP@0.5:0.95     : {map_50_95:.3f}")


Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1531.8±1118.8 MB/s, size: 45.7 KB)


val: Scanning /content/yolo_dataset/validation/labels.cache... 332 images, 11 backgrounds, 0 corrupt: 100%|██████████| 332/332 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 6, len(boxes) = 412. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:04<00:00,  4.36it/s]


                   all        332        412      0.909      0.908      0.945      0.776
                Laptop         55         58       0.93      0.916       0.98      0.924
                Bottle         79        109      0.933      0.945       0.98      0.754
                 Mouse        100        134      0.912      0.846       0.95      0.812
                 Watch         76        100      0.807      0.836      0.821      0.569
              Ishpreet         11         11      0.965          1      0.995      0.823
Speed: 1.1ms preprocess, 3.7ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/val3

📊 Evaluation Metrics:
Mean Precision   : 0.909
Mean Recall      : 0.908
mAP@0.5          : 0.945
mAP@0.5:0.95     : 0.776
